## Housing Prices and Park Proximity

#### Introduction

This project will examine the relationship between housing prices and parks.  Real estate professionals are always looking for factors that influence prices.  It is a cliche that the most important thing in real estate is location, location, location.  I hope to examine how the location of a house relative to popular parks may impact pricing.  This project would help real estate agents, real estate appraisers, single family rental investors, and prospective buyers and sellers in understanding the value of a particular house. In addition, the project could help city planners with park placement and management.

Parks, particularly popular ones, are attractive features for a neighborhood.  They provide space for a variety of outdoor activities including excercise, picnics, and enjoying nature.  While parks are not typically a feature that prospective house buyers look for, they provide real benefits when they are nearby and well maintained.  I hope to examine the impact that popular parks have on housing prices in King County, Washington.  My hypothesis is that popular parks would increase the value of houses in close proximity.

#### Data

In a previous class we examined the housing data for King County, Washington and how various features of the house itself impacted prices.  The housing data from that project included the latitude and longitude of the houses as well as the prices.  Foursquare provides information on a variety of venues including parks.  I will retrieve the location of the popular parks in King County, Washington from Foursquare.  

Using the latitude and longitude of both the parks and houses in the two data sets we can calculate the distance each house is from a particular park.  We will use the popularity of a park on Foursquare as a measure of those that are attractive to potential buyers.  With distance from popular parks and house price data we should be able to determine if proximity has an impact on housing prices.

###### Housing Price Data:

In [3]:
import pandas as pd

file_name='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/coursera/project/kc_house_data_NaN.csv'
df_stlhouse=pd.read_csv(file_name)
df_stlhouse.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,7129300520,20141013T000000,221900.0,3.0,1.00,1180,5650,1.0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,6414100192,20141209T000000,538000.0,3.0,2.25,2570,7242,2.0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,5631500400,20150225T000000,180000.0,2.0,1.00,770,10000,1.0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,3,2487200875,20141209T000000,604000.0,4.0,3.00,1960,5000,1.0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,4,1954400510,20150218T000000,510000.0,3.0,2.00,1680,8080,1.0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


###### King County Park Data:

In [25]:
from geopy.geocoders import Nominatim

latlist = list()
lonlist = list()

zcode = df_stlhouse['zipcode'].unique()
for z in zcode:
    address = z
    geolocator = Nominatim(user_agent="stl_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    latlist.append(latitude)
    lonlist.append(longitude)
df_ll=pd.DataFrame(list(zip(latlist,lonlist)),columns=['Latitude','Longitude'])
df_ll['Zipcode'] = zcode
df_ll.head()

,Latitude,Longitude,Zipcode
0,47.498639,-122.247645,98178
1,47.721468,-122.305411,98125
2,47.756876,-122.241875,98028
3,38.099476,15.495463,98136
4,47.614466,-122.035507,98074


In [30]:
# create a map of King County Zipcodes
import folium

address = 'King County, WA'
geolocator = Nominatim(user_agent="stl_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_stl = folium.Map(location=[latitude, longitude], zoom_start=9)

for lat, lng, zcode in zip(df_ll['Latitude'], df_ll['Longitude'], df_ll['Zipcode']):
    label = zcode
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stl)  
    
map_stl

In [27]:
# @hidden_cell
CLIENT_ID = 'PYRV4LPYOCPHX2OUBUPCOSNKD4DLATNPETUIGRDWGK3TQG23' 
CLIENT_SECRET = 'WRFCZFZEFBF5C2RWWGJ1LMNIIAIJM2XO45AXC0FQZACK1LJI' 
VERSION = '20180605' 

In [41]:
# create request to get parks in and near King County, WA and save as json file
import requests
import json
from pandas.io.json import json_normalize

query = 'park'
            
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}&limit={}&sortByPopularity{}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    40000,
    query,
    100,
    1)
            
results = requests.get(url).json()['response']['groups'][0]['items']

df_parks = json_normalize(results)
df_parks.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,...,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.venuePage.id,venue.location.neighborhood
0,e-0-4c3e2833ca012d7fce172fbe-0,0,"[{'summary': 'This spot is popular', 'type': '...",4c3e2833ca012d7fce172fbe,Snoqualmie Point Park,37198 Winery Rd,47.510243,-121.841994,"[{'label': 'display', 'lat': 47.51024308065131...",2968,...,Snoqualmie Valley,WA,United States,"[37198 Winery Rd, Snoqualmie Valley, WA 98045,...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN
1,e-0-4b773e3ef964a520188b2ee3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4b773e3ef964a520188b2ee3,Tiger Mountain Summit,11900 Issaquah Hobart Rd SE,47.444143,-121.961476,"[{'label': 'display', 'lat': 47.44414339220613...",8883,...,Issaquah,WA,United States,[11900 Issaquah Hobart Rd SE (SE 113th Street)...,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],SE 113th Street,NaN,NaN
2,e-0-540b8cf8498ee75cf32d4fdd-2,0,"[{'summary': 'This spot is popular', 'type': '...",540b8cf8498ee75cf32d4fdd,Jeanne Hansen Park,34226 SE Jacobia Street,47.518542,-121.881771,"[{'label': 'display', 'lat': 47.5185420875111,...",3772,...,Snoqualmie,WA,United States,"[34226 SE Jacobia Street, Snoqualmie, WA 98065...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN
3,e-0-463fd072f964a52051461fe3-3,0,"[{'summary': 'This spot is popular', 'type': '...",463fd072f964a52051461fe3,Gene Coulon Park,1201 Lake Washington Blvd N,47.506602,-122.201966,"[{'label': 'display', 'lat': 47.50660166897992...",25642,...,Renton,WA,United States,"[1201 Lake Washington Blvd N, Renton, WA 98056...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN
4,e-0-439158cdf964a520572b1fe3-4,0,"[{'summary': 'This spot is popular', 'type': '...",439158cdf964a520572b1fe3,Marymoor Park,6046 W Lake Sammamish Pkwy NE,47.662653,-122.118050,"[{'label': 'display', 'lat': 47.66265318016147...",27386,...,Redmond,WA,United States,"[6046 W Lake Sammamish Pkwy NE, Redmond, WA 98...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[],NaN,NaN,NaN


In [43]:
df_parks = df_parks[['venue.name','venue.location.address','venue.location.city','venue.location.lat','venue.location.lng']]
df_parks.head()

,venue.name,venue.location.address,venue.location.city,venue.location.lat,venue.location.lng
0,Snoqualmie Point Park,37198 Winery Rd,Snoqualmie Valley,47.510243,-121.841994
1,Tiger Mountain Summit,11900 Issaquah Hobart Rd SE,Issaquah,47.444143,-121.961476
2,Jeanne Hansen Park,34226 SE Jacobia Street,Snoqualmie,47.518542,-121.881771
3,Gene Coulon Park,1201 Lake Washington Blvd N,Renton,47.506602,-122.201966
4,Marymoor Park,6046 W Lake Sammamish Pkwy NE,Redmond,47.662653,-122.118050


In [45]:
df_parks.rename(columns={'venue.name':'Park_Name','venue.location.address':'Address','venue.location.city':'City','venue.location.lat':'Latitude','venue.location.lng':'Longitude'},inplace=True)
df_parks.head()

,Park_Name,Address,City,Latitude,Longitude
0,Snoqualmie Point Park,37198 Winery Rd,Snoqualmie Valley,47.510243,-121.841994
1,Tiger Mountain Summit,11900 Issaquah Hobart Rd SE,Issaquah,47.444143,-121.961476
2,Jeanne Hansen Park,34226 SE Jacobia Street,Snoqualmie,47.518542,-121.881771
3,Gene Coulon Park,1201 Lake Washington Blvd N,Renton,47.506602,-122.201966
4,Marymoor Park,6046 W Lake Sammamish Pkwy NE,Redmond,47.662653,-122.118050
